In [57]:
from IPython.core.display import HTML
HTML("<style>.container { width:100% !important; }</style>")

In [58]:
%run initialize_notebook.ipynb

In [59]:
from pycountry import countries
from constructive_geometries import Geomatcher
from wurst import *
from wurst.ecoinvent.electricity_markets import *
from wurst.searching import *
from wurst.ecoinvent.filters import *
import os.path

# Functions to clean up Wurst import and additional technologies

In [60]:
def fix_unset_technosphere_and_production_exchange_locations(db, matching_fields=('name', 'unit')):
    for ds in db:
        for exc in ds['exchanges']:
            if exc['type'] == 'production' and exc.get('location') is None:
                exc['location'] = ds['location']                
            elif exc['type'] == 'technosphere' and exc.get('location') is None:
                locs = find_location_given_lookup_dict(db, 
                                                       {k: exc.get(k) for k in matching_fields})
                if len(locs) == 1:
                    exc['location'] = locs[0]
                else:
                    print("No unique location found for exchange:\n{}\nFound: {}".format(
                        pprint.pformat(exc), locs
                    ))
                        
def find_location_given_lookup_dict(db, lookup_dict):
    return [x['location'] for x in get_many(db, *[equals(k, v) for k, v in lookup_dict.items()])]

In [61]:
exists = lambda x: {k: v for k, v in x.items() if v is not None}

def remove_nones(db):
    for ds in db:
        ds['exchanges'] = [exists(exc) for exc in ds['exchanges']]

In [62]:
def set_global_location_for_additional_datasets(db):
        """ This function is needed because the wurst function relink_technosphere exchanges needs global datasets if if can't find a regional one."""    
        non_ecoinvent_datasets = [x['name'] for x in db  if x['database'] not in ['ecoinvent', 'ecoinvent_unchanged']]
        ecoinvent_datasets = [x['name'] for x in db  if x['database'] not in ['ecoinvent', 'ecoinvent_unchanged']]

        for ds in [x for x in db if x['database'] in ['Carma CCS', 'CSP']]:
            print('Dataset: ',ds['name'], ds['location'], ' Changing to Global')
            ds['location'] = 'GLO'
            for exc in [x for x in ds['exchanges'] if x['type'] != 'biosphere']:
                if exc['name'] in non_ecoinvent_datasets:
                    if exc['name'] in ecoinvent_datasets and exc['location'] != 'GLO': print ('Ecoinvent exchange: ',exc['name'], exc['location'])
                    else: 
                        print('Exchange: ',exc['name'], exc['location'], 'Changing to Global')
                        exc['location'] = 'GLO'

# Region and location mapping

In [63]:
#these locations aren't found correctly by the constructive geometries library - we correct them here:
fix_names= {#'CSG' : 'CN-CSG',
            #'SGCC': 'CN-SGCC',
             
             #'RFC' : 'US-RFC',
             #'SERC' : 'US-SERC',
             #'TRE': 'US-TRE',
             #'ASCC': 'US-ASCC',
             #'HICC': 'US-HICC',
             #'FRCC': 'US-FRCC',
             #'SPP' : 'US-SPP',
             #'MRO, US only' : 'US-MRO', 
             #'NPCC, US only': 'US-NPCC', 
             #'WECC, US only': 'US-WECC',
             
             'IAI Area, Africa':'IAI Area 1, Africa',
             'IAI Area, South America':'IAI Area 3, South America', 
             'IAI Area, Asia, without China and GCC':'IAI Area 4&5, without China', 
             'IAI Area, North America, without Quebec':'IAI Area 2, without Quebec',
             'IAI Area, Gulf Cooperation Council':'IAI Area 8, Gulf'
            }

fix_names_back = {v:k for k,v in fix_names.items()}


def rename_locations(db, name_dict):
    for ds in db:
        if ds['location'] in name_dict:
            ds['location'] = name_dict[ds['location']]

        for exc in technosphere(ds):
            if exc['location'] in name_dict:
                exc['location'] = name_dict[exc['location']]
    


In [64]:
def get_remind_geomatcher():
    """Return geomatcher object which includes REMIND regions."""

    regionmapping = pd.read_csv("regionmappingH12.csv", sep=";")    
    iso_2_rmnd = regionmapping.set_index("CountryCode").to_dict()["RegionCode"]
    rmnd_2_iso = regionmapping.groupby("RegionCode")["CountryCode"].apply(list).to_dict()

    geomatcher = Geomatcher()

    # cc (cocos islands) seem to be not there, skipping
    # list(geomatcher['CX'])[:10]
    not_found = ["CCK", "CXR", 'GGY', 'JEY', 'BLM', 'MAF']
    rmnd_2_iso_fix = {rmnd: [iso for iso in rmnd_2_iso[rmnd] if iso not in not_found] for rmnd in rmnd_2_iso}
    #_rmnd_2_iso2_fix = {rmnd: [iso for iso in _rmnd_2_iso2[rmnd] if iso != "CC"] for rmnd in _rmnd_2_iso2}

    geomatcher.add_definitions(rmnd_2_iso_fix, "REMIND")
    return geomatcher

if [x for x in geomatcher if 'REMIND' in x] != [('REMIND', 'CAZ'),
                                                ('REMIND', 'CHA'),
                                                ('REMIND', 'EUR'),
                                                ('REMIND', 'IND'),
                                                ('REMIND', 'JPN'),
                                                ('REMIND', 'LAM'),
                                                ('REMIND', 'MEA'),
                                                ('REMIND', 'NEU'),
                                                ('REMIND', 'OAS'),
                                                ('REMIND', 'REF'),
                                                ('REMIND', 'SSA'),
                                                ('REMIND', 'USA')]:
    geomatcher = get_remind_geomatcher()

Geomatcher: Used 'AU' for 'AUS'
Geomatcher: Used 'CA' for 'CAN'
Geomatcher: Used 'HM' for 'HMD'
Geomatcher: Used 'NZ' for 'NZL'
Geomatcher: Used 'PM' for 'SPM'
Geomatcher: Used 'CN' for 'CHN'
Geomatcher: Used 'HK' for 'HKG'
Geomatcher: Used 'MO' for 'MAC'
Geomatcher: Used 'TW' for 'TWN'
Geomatcher: Used 'AX' for 'ALA'
Geomatcher: Used 'AT' for 'AUT'
Geomatcher: Used 'BE' for 'BEL'
Geomatcher: Used 'BG' for 'BGR'
Geomatcher: Used 'CY' for 'CYP'
Geomatcher: Used 'CZ' for 'CZE'
Geomatcher: Used 'DE' for 'DEU'
Geomatcher: Used 'DK' for 'DNK'
Geomatcher: Used 'ES' for 'ESP'
Geomatcher: Used 'EE' for 'EST'
Geomatcher: Used 'FI' for 'FIN'
Geomatcher: Used 'FR' for 'FRA'
Geomatcher: Used 'FO' for 'FRO'
Geomatcher: Used 'GB' for 'GBR'
Geomatcher: Used 'GI' for 'GIB'
Geomatcher: Used 'GR' for 'GRC'
Geomatcher: Used 'HR' for 'HRV'
Geomatcher: Used 'HU' for 'HUN'
Geomatcher: Used 'IM' for 'IMN'
Geomatcher: Used 'IE' for 'IRL'
Geomatcher: Used 'IT' for 'ITA'
Geomatcher: Used 'LT' for 'LTU'
Geomatch

In [65]:
def ecoinvent_to_remind_locations(loc):
    if loc== 'RoW':
        loc='GLO'
    
    if loc in fix_names.keys():
        loc = fix_names[loc]
        
    if loc == 'IAI Area, Russia & RER w/o EU27 & EFTA':
        loc = 'RU'
        
    remind_loc = [r[1] for r in geomatcher.intersects(loc) if r[0]=='REMIND']
    
    ei_35_new_locs = {'XK':['NEU']}
    
    if not remind_loc:
        if loc in ei_35_new_locs:
            remind_loc =  ei_35_new_locs[loc]
        else: 
            print('No location found for: ' + loc)
            return None
    return remind_loc

# Import Remind data

In [66]:
def get_remind_data(scenario_name):
    #This file reads the REMIND csv result file and returns a dataframe containing all the information
    file_name = os.path.join("Remind output files", scenario_name+ ".mif")           
    df = pd.read_csv(file_name,sep=';', index_col = ['Region', 'Variable', 'Unit']).drop(columns = ['Model', 'Scenario', 'Unnamed: 24'])
    df.columns =df.columns.astype(int)

    return df

# Documenting changes to ecoinvent

In [67]:
def get_exchange_amounts(ds, technosphere_filters=None, biosphere_filters=None):
    result={}
    for exc in technosphere(ds, *(technosphere_filters or [])):
        result[(exc['name'], exc['location'])]=exc['amount']
    for exc in biosphere(ds, *(biosphere_filters or [])):
        result[(exc['name'], exc['categories'])]=exc['amount']
    return result

# Modify electricity markets

## Import remind electricity markets

In [68]:
remind_electricity_market_labels = {

 'Biomass CHP': 'SE|Electricity|Biomass|CHP|w/o CCS',
 'Biomass IGCC CCS': 'SE|Electricity|Biomass|IGCCC|w/ CCS',
 'Biomass IGCC': 'SE|Electricity|Biomass|IGCC|w/o CCS',

 'Coal PC': 'SE|Electricity|Coal|PC|w/o CCS',
 'Coal IGCC': 'SE|Electricity|Coal|IGCC|w/o CCS',
 'Coal PC CCS': 'SE|Electricity|Coal|PCC|w/ CCS',
 'Coal IGCC CCS': 'SE|Electricity|Coal|IGCCC|w/ CCS',
 'Coal CHP':'SE|Electricity|Coal|CHP|w/o CCS',

'Gas OC': 'SE|Electricity|Gas|GT',
'Gas CC':'SE|Electricity|Gas|CC|w/o CCS',
'Gas CHP': 'SE|Electricity|Gas|CHP|w/o CCS',
'Gas CCS':  'SE|Electricity|Gas|w/ CCS',
    
 'Geothermal': 'SE|Electricity|Geothermal',
    
 'Hydro': 'SE|Electricity|Hydro',
    
 'Hydrogen': 'SE|Electricity|Hydrogen',
    
 'Nuclear': 'SE|Electricity|Nuclear',
    
 'Oil': 'SE|Electricity|Oil|w/o CCS',
    
 'Solar CSP': 'SE|Electricity|Solar|CSP',
 'Solar PV': 'SE|Electricity|Solar|PV',
    
 'Wind': 'SE|Electricity|Wind',
}

rename_remind_electricity_market_labels = {v:k for k, v in remind_electricity_market_labels.items()}

In [69]:
def get_remind_markets(remind_data, year, drop_hydrogen=True):
    if year < 2005 or year >2150:
        print('year not valid, must be between 2005 and 2100')
        return
    
    elif year in remind_data.columns:
        result =  remind_data.unstack(level=0)[year].loc[list(remind_electricity_market_labels.values())].reset_index(level=1, drop=True).rename(index = rename_remind_electricity_market_labels).divide(
            remind_data.unstack(level=0)[year].loc[list(remind_electricity_market_labels.values())].sum(axis=0)).drop('World', axis=1)

    else: 
        temp = remind_data.unstack(level=0).loc[list(remind_electricity_market_labels.values())].reset_index(level=1, drop=True).rename(index = rename_remind_electricity_market_labels).stack(level=1).T
        new = pd.DataFrame(index = temp.columns,columns = [year],  data = np.nan).T
    
        result =  pd.concat([temp, new]).sort_index().interpolate(method = 'values').loc[year].unstack(level=1)
        
    if drop_hydrogen == False:
        return result
    else: 
        print('Excluding hydrogen from electricity markets.\nHydrogen had a maximum share of '+ str(round(result.loc['Hydrogen'].max() * 100, 2)) + ' %')
        return result.drop('Hydrogen', axis = 0).divide(result.drop('Hydrogen', axis = 0).sum())

## Define technology matching between remind and ecoinvent

In [70]:
available_electricity_generating_technologies={

    #From Carma project
    'Biomass IGCC CCS':['Electricity, from CC plant, 100% SNG, truck 25km, post, pipeline 200km, storage 1000m/2025',
                    'Electricity, at wood burning power plant 20 MW, truck 25km, post, pipeline 200km, storage 1000m/2025',
                    'Electricity, at BIGCC power plant 450MW, pre, pipeline 200km, storage 1000m/2025'],
    
    #From Carma project
    'Biomass IGCC': ['Electricity, at BIGCC power plant 450MW, no CCS/2025'],
    
    # only Biomass CHP available
    'Biomass CHP':['heat and power co-generation, wood chips, 6667 kW, state-of-the-art 2014', 
                    'heat and power co-generation, wood chips, 6667 kW',
                    'heat and power co-generation, biogas, gas engine'],
    

    #From Carma project
    'Coal IGCC':['Electricity, at power plant/hard coal, IGCC, no CCS/2025', 
                'Electricity, at power plant/lignite, IGCC, no CCS/2025'],
    
    'Coal IGCC CCS':['Electricity, at power plant/hard coal, pre, pipeline 200km, storage 1000m/2025',
                      'Electricity, at power plant/lignite, pre, pipeline 200km, storage 1000m/2025',],
   
    'Coal PC':['electricity production, hard coal',
                'electricity production, lignite'
              'electricity production, hard coal, conventional',
              'electricity production, hard coal, supercritical'],
                
    'Coal CHP': ['heat and power co-generation, hard coal',
                'heat and power co-generation, lignite'],
    
    #From Carma project
     'Coal PC CCS':[ 'Electricity, at power plant/hard coal, post, pipeline 200km, storage 1000m/2025',
                 'Electricity, at power plant/lignite, post, pipeline 200km, storage 1000m/2025'], 

    
    #From Carma project
    'Gas CCS':['Electricity, at power plant/natural gas, pre, pipeline 200km, storage 1000m/2025',
                        'Electricity, at power plant/natural gas, post, pipeline 200km, storage 1000m/2025'],
    
    'Gas OC':['electricity production, natural gas, conventional power plant'],
            
    'Gas CC': ['electricity production, natural gas, combined cycle power plant'],    
     
    'Gas CHP': ['heat and power co-generation, natural gas, combined cycle power plant, 400MW electrical',
            'heat and power co-generation, natural gas, conventional power plant, 100MW electrical'],
            
    'Geothermal':['electricity production, deep geothermal'],
    
    'Hydro':['electricity production, hydro, reservoir, alpine region',
            'electricity production, hydro, reservoir, non-alpine region',
            'electricity production, hydro, reservoir, tropical region',
            'electricity production, hydro, run-of-river'],
    
    'Hydrogen':[],
    
    'Nuclear':['electricity production, nuclear, boiling water reactor',
                'electricity production, nuclear, pressure water reactor, heavy water moderated',
                'electricity production, nuclear, pressure water reactor'],
    
    'Oil':['electricity production, oil',
          'heat and power co-generation, oil'],
    
    'Solar CSP': ['electricity production, solar thermal parabolic trough, 50 MW', 
               'electricity production, solar tower power plant, 20 MW'],
    
    'Solar PV':['electricity production, photovoltaic, 3kWp slanted-roof installation, multi-Si, panel, mounted',
                    'electricity production, photovoltaic, 3kWp slanted-roof installation, single-Si, panel, mounted',
                    'electricity production, photovoltaic, 570kWp open ground installation, multi-Si'], 
    
    'Wind':['electricity production, wind, <1MW turbine, onshore',
            'electricity production, wind, 1-3MW turbine, onshore',
            'electricity production, wind, >3MW turbine, onshore',
            'electricity production, wind, 1-3MW turbine, offshore']
}

## Functions for modifying ecoinvent electricity markets

In [71]:
electricity_market_filter_high_voltage= [contains('name', 'market for electricity, high voltage'),
                                doesnt_contain_any('name', ['aluminium industry','internal use in coal mining'])]
    
electricity_market_filter_medium_voltage= [contains('name', 'market for electricity, medium voltage'),
                                doesnt_contain_any('name', ['aluminium industry','electricity, from municipal waste incineration'])]
    
electricity_market_filter_low_voltage= [contains('name', 'market for electricity, low voltage')]

In [72]:
def delete_electricity_inputs_from_market(ds):
    #This function reads through an electricity market dataset and deletes all electricity inputs that are not own consumption. 
    ds['exchanges'] = [exc for exc in get_many(ds['exchanges'], *[either(*[exclude(contains('unit', 'kilowatt hour')),
                                                                           contains('name', 'market for electricity, high voltage'),
                                                                           contains('name', 'market for electricity, medium voltage'),
                                                                           contains('name', 'market for electricity, low voltage'),
                                                                           contains('name', 'electricity voltage transformation')])])]

In [73]:
def find_average_mix(df):
    #This function considers that there might be several remind regions that match the ecoinvent region. This function returns the average mix across all regions.
    return df.mean(axis=1).divide(df.mean(axis=1).sum())

In [74]:
def find_ecoinvent_electricity_datasets_in_same_ecoinvent_location(tech, location, db):
    #first try ecoinvent location code:
    try: return [x for x in get_many(db, *[either(*[equals('name', name) for name in available_electricity_generating_technologies[tech]]), 
                                            equals('location', location), equals('unit', 'kilowatt hour')])]
    #otherwise try remind location code (for new datasets)
    except:
        try: return [x for x in get_many(db, *[either(*[equals('name', name) for name in available_electricity_generating_technologies[tech]]), 
                                            equals('location', ecoinvent_to_remind_locations(location)), equals('unit', 'kilowatt hour')])]
        except: return []

In [75]:
def find_other_ecoinvent_regions_in_remind_region(loc):
    if loc== 'RoW':
        loc='GLO'
    
    if loc in fix_names:
        loc = fix_names[loc]
    
    remind_regions = [r for r in geomatcher.intersects(loc) if r[0]=='REMIND']

    temp = []
    for remind_region in remind_regions:
        temp.extend([r for r in geomatcher.contained(remind_region)])

    result = []
    for temp in temp:
        if type(temp) ==tuple:
            result.append(temp[1])
        else: result.append(temp)
    return set(result)

In [76]:
def find_ecoinvent_electricity_datasets_in_remind_location(tech, location, db):
    try: return [x for x in get_many(db, *[either(*[equals('name', name) for name in available_electricity_generating_technologies[tech]]), 
                                            either(*[equals('location', loc) for loc in find_other_ecoinvent_regions_in_remind_region(location)]),
                                        equals('unit', 'kilowatt hour')
                          ])]
    except: return []

In [77]:
def find_ecoinvent_electricity_datasets_in_all_locations(tech, db):
       return [x for x in get_many(db, *[either(*[equals('name', name) for name in available_electricity_generating_technologies[tech]]),equals('unit', 'kilowatt hour')])]

In [78]:
def add_new_datasets_to_electricity_market(ds, db, remind_electricity_market_df, year):
    #This function adds new electricity datasets to a market based on remind results. We pass not only a dataset to modify, but also a pandas dataframe containing the new electricity mix information, and the db from which we should find the datasets
    # find out which remind regions correspond to our dataset:
        
    remind_locations= ecoinvent_to_remind_locations(ds['location'])
    
    # here we find the mix of technologies in the new market and how much they contribute:
    mix =  find_average_mix(remind_electricity_market_df[remind_locations]) #could be several remind locations - we just take the average
   
    
    # here we find the datasets that will make up the mix for each technology
    datasets={}
    for i in mix.index:
        if mix[i] !=0:
            
            #print('Next Technology: ',i) 
            
            # We have imports defined for Switzerland. Let's do those first:
            if i == 'Imports':
                datasets[i] = [x for x in get_many(db, *[equals('name', 'market group for electricity, high voltage'), equals('location', 'ENTSO-E')])]
            else:
                # First try to find a dataset that is from that location (or remind region for new datasets):   
                datasets[i] = find_ecoinvent_electricity_datasets_in_same_ecoinvent_location(i, ds['location'], db)
                #print('First round: ',i, [(ds['name'], ds['location']) for ds in datasets[i]])
            
                #If this doesn't work, we try to take a dataset from another ecoinvent region within the same remind region                                    
                if len(datasets[i]) == 0: 
                    datasets[i] = find_ecoinvent_electricity_datasets_in_remind_location(i, ds['location'], db)
                    #print('Second round: ',i, [(ds['name'], ds['location']) for ds in datasets[i]])
            
                # If even this doesn't work, try taking a global datasets 
                if len(datasets[i]) == 0:  
                    datasets[i] = find_ecoinvent_electricity_datasets_in_same_ecoinvent_location(i, 'GLO', db)
                    #print('Third round: ',i, [(ds['name'], ds['location']) for ds in datasets[i]])
                    
                #if no global dataset available, we just take the average of all datasets we have:
                if len(datasets[i]) ==0:  
                    datasets[i] = find_ecoinvent_electricity_datasets_in_all_locations(i, db)
                    #print('Fourth round: ',i, [(ds['name'], ds['location']) for ds in datasets[i]])
                
            #If we still can't find a dataset, we just take the global market group
            if len(datasets[i]) ==0:
                print('No match found for location: ', ds['location'], ' Technology: ', i,'. Taking global market group for electricity')
                datasets[i] = [x for x in get_many(db, *[equals('name', 'market group for electricity, high voltage'), equals('location', 'GLO')])]
                                            
                            
    # Now we add the new exchanges:
    for i in mix.index:
        if mix[i] !=0:
            total_amount = mix[i]
            amount= total_amount / len(datasets[i])
            for dataset in datasets[i]:
                ds['exchanges'].append({
                'amount': amount,
                'unit': dataset['unit'],    
                'input': (dataset['database'], dataset['code']),
                'type': 'technosphere',
                'name': dataset['name'],
                'location': dataset['location']                           
                    })
    
    #confirm that exchanges sum to 1!
    sum = np.sum([exc['amount'] for exc in technosphere(ds, *[equals('unit', 'kilowatt hour'), doesnt_contain_any('name', ['market for electricity, high voltage'])])])
    if round(sum,4) != 1.00:  print(ds['location'], " New exchanges don't add to one! something is wrong!", sum )
    return

In [79]:
def update_electricity_markets(db, year, remind_data):
    
    #import the remind market mix from the remind result files:
    remind_electricity_market_df = get_remind_markets(remind_data, year, drop_hydrogen=True)

    #Remove all electricity producers from markets:
    db = empty_low_voltage_markets(db)
    db = empty_medium_voltage_markets(db)
    db = empty_high_voltage_markets(db) # This function isn't working as expected - it needs to delete imports as well.
    
    changes={}
    #update high voltage markets:
    for ds in get_many(db, *electricity_market_filter_high_voltage):
        changes[ds['code']]={}
        changes[ds['code']].update( {('meta data', x) : ds[x] for x in ['name','location']})
        changes[ds['code']].update( {('original exchanges', k) :v for k,v in get_exchange_amounts(ds).items()})
        delete_electricity_inputs_from_market(ds) # This function will delete the markets. Once Wurst is updated this can be deleted.
        add_new_datasets_to_electricity_market(ds, db, remind_electricity_market_df, year)
        changes[ds['code']].update( {('updated exchanges', k) :v for k,v in get_exchange_amounts(ds).items()})
    return changes

# Modify Fossil Electricity Generation Technologies

## get remind technology efficiencies

In [80]:
def get_remind_fossil_electricity_efficiency(remind_data, year, technology):

    
    fossil_electricity_efficiency_name_dict = {
     'Biomass IGCC CCS':'Tech|Electricity|Biomass|IGCCC|w/ CCS|Efficiency', 
     'Biomass CHP':     'Tech|Electricity|Biomass|CHP|w/o CCS|Efficiency', 
     'Biomass IGCC':    'Tech|Electricity|Biomass|IGCC|w/o CCS|Efficiency',
        
    'Coal IGCC':'Tech|Electricity|Coal|IGCC|w/o CCS|Efficiency', 
     'Coal IGCC CCS':'Tech|Electricity|Coal|IGCCC|w/ CCS|Efficiency',
     'Coal PC':'Tech|Electricity|Coal|PC|w/o CCS|Efficiency', 
     'Coal PC CCS':'Tech|Electricity|Coal|PCC|w/ CCS|Efficiency', 
     'Coal CHP': 'Tech|Electricity|Coal|CHP|w/o CCS|Efficiency',
        
    'Gas OC':'Tech|Electricity|Gas|GT|Efficiency', 
     'Gas CC':'Tech|Electricity|Gas|CC|w/o CCS|Efficiency',
     'Gas CHP': 'Tech|Electricity|Gas|CHP|w/o CCS|Efficiency',
     'Gas CCS':'Tech|Electricity|Gas|CCC|w/ CCS|Efficiency',
     
    'Oil':'Tech|Electricity|Oil|DOT|Efficiency',
    }
    
    if year < 2005 or year > 2150:
        print('year not valid, must be between 2005 and 2150')
        return    
    if technology not in fossil_electricity_efficiency_name_dict:
        print("Technology name not recognized: {}".format(technology))
        return
    
    if fossil_electricity_efficiency_name_dict[technology] not in remind_data.index.levels[1]: 
        print('Technology efficiency not in REMIND output file: {}'.format(technology))
        return
    
    
    
    elif year in remind_data.columns:
        result =  remind_data.unstack(level=0).loc[fossil_electricity_efficiency_name_dict[technology]].stack(level = 0).reset_index(level = 0, drop=True).loc[year]
    else: 
        temp = remind_data.unstack(level=0).loc[fossil_electricity_efficiency_name_dict[technology]].stack(level = 0).reset_index(level = 0, drop=True)
        new = pd.DataFrame(index = temp.columns,columns = [year],  data = np.nan).T
        result =  pd.concat([temp, new]).sort_index().interpolate(method = 'values').loc[year]
    
    if 0 in result.values:  
        print('Warning: technology has regions with zero efficiency: {}'.format(technology))
        print(result)
        
    return result

## get ecoinvent efficiencies

In [81]:
def find_ecoinvent_coal_efficiency(ds):
    # Nearly all coal power plant datasets have the efficiency as a parameter. 
    # If this isn't available, we back calculate it using the amount of coal used and 
    # an average energy content of coal.
    try: 
        return ds['parameters']['efficiency']
    except KeyError:
        pass
    
    #print('Efficiency parameter not found - calculating generic coal efficiency factor', ds['name'], ds['location'])
    
    fuel_sources = technosphere(ds, 
                                either(contains('name', 'hard coal'), contains('name', 'lignite')), 
                                doesnt_contain_any('name', ('ash','SOx')),
                                equals('unit', 'kilogram'))
    energy_in = 0 
    for exc in fuel_sources:
        if 'hard coal' in exc['name']: 
            energy_density = 20.1 / 3.6 #kWh/kg
        elif 'lignite' in exc['name']: 
            energy_density = 9.9 / 3.6 # kWh/kg
        else:
            raise ValueError("Shouldn't happen because of filters!!!")
        energy_in += (exc['amount'] * energy_density)
    ds['parameters']['efficiency'] = reference_product(ds)['amount'] / energy_in
    #print(ds['parameters']['efficiency'])
    return reference_product(ds)['amount'] / energy_in

In [82]:
def find_ecoinvent_gas_efficiency(ds):
    
    #Nearly all gas power plant datasets have the efficiency as a parameter. 
    #If this isn't available, we back calculate it using the amount of gas used and an average energy content of gas.
    try: 
        return ds['parameters']['efficiency']
    except KeyError:
        pass
    
    #print('Efficiency parameter not found - calculating generic gas efficiency factor', ds['name'], ds['location'])
    
    fuel_sources = technosphere(ds,
                                either(contains('name', 'natural gas, low pressure'), contains('name', 'natural gas, high pressure')), 
                                equals('unit', 'cubic meter'))
    energy_in = 0 
    for exc in fuel_sources:
        #(based on energy density of natural gas input for global dataset 'electricity production, natural gas, conventional power plant')
        if 'natural gas, high pressure' in exc['name']: 
            energy_density= 39 / 3.6 # kWh/m3 
        
        #(based on average energy density of high pressure gas, scaled by the mass difference listed between high pressure and low pressure gas in the dataset: 
        #natural gas pressure reduction from high to low pressure, RoW)
        elif 'natural gas, low pressure' in exc['name']: energy_density= 39 * 0.84 / 3.6 #kWh/m3 
        else:
            raise ValueError("Shouldn't happen because of filters!!!")
        energy_in += (exc['amount'] * energy_density)
    ds['parameters']['efficiency'] = reference_product(ds)['amount'] / energy_in
    #print(ds['parameters']['efficiency'])
    return reference_product(ds)['amount'] / energy_in


In [83]:
def find_ecoinvent_oil_efficiency(ds):
    
    #Nearly all oil power plant datasets have the efficiency as a parameter. If this isn't available, we use global average values to calculate it.
    try: return ds['parameters']['efficiency_oil_country']
    except KeyError:
        pass
    #print('Efficiency parameter not found - calculating generic oil efficiency factor', ds['name'], ds['location'])
    fuel_sources=[x for x in technosphere(ds, *[contains('name', 'heavy fuel oil'), 
                                    equals('unit', 'kilogram')]
                                    )]
    energy_in=0 
    for exc in fuel_sources:
        #(based on energy density of heavy oil input and efficiency parameter for dataset 'electricity production, oil, RoW')
        energy_density= 38.5 / 3.6 # kWh/m3 
        energy_in += (exc['amount'] * energy_density)
    ds['parameters']['efficiency'] = reference_product(ds)['amount'] / energy_in
    #print(ds['parameters']['efficiency'])
    return reference_product(ds)['amount'] /energy_in

In [84]:
def find_ecoinvent_biomass_efficiency(ds):
    #Nearly all power plant datasets have the efficiency as a parameter. If this isn't available, we excl.
    try: return ds['parameters']['efficiency_electrical']
    except: pass
    
    if ds['name'] == 'heat and power co-generation, biogas, gas engine, label-certified': 
        ds['parameters'] = {'efficiency_electrical': 0.32}
        return ds['parameters']['efficiency_electrical']#in general comments for dataset
    
    elif ds['name'] == 'wood pellets, burned in stirling heat and power co-generation unit, 3kW electrical, future': 
        ds['parameters'] = {'efficiency_electrical': 0.23}
        return ds['parameters']['efficiency_electrical'] #in comments for dataset  
    
    print(ds['name'], ds['location'],' Efficiency not found!')
    return 0

In [85]:
def update_ecoinvent_efficiency_parameter(ds, scaling_factor):
    parameters = ds['parameters']
    possibles = ['efficiency', 'efficiency_oil_country', 'efficiency_electrical']

    for key in possibles:
        try: 
            parameters[key] /= scaling_factor
            return
        except KeyError:   
            pass

## Find efficiency scaling factors:

In [86]:
def find_coal_efficiency_scaling_factor(ds, year, remind_efficiency, agg_func=np.average):
    #input a coal electricity dataset and year. We look up the efficiency for this region and year from the remind model and return the scaling factor by which to multiply all efficiency dependent exchanges.
    #If the ecoinvent region corresponds to multiple remind regions we simply average them.
    ecoinvent_eff = find_ecoinvent_coal_efficiency(ds)
    remind_locations= ecoinvent_to_remind_locations(ds['location'])
    remind_eff = agg_func(remind_efficiency [remind_locations].values)/100 # we take an average of all applicable remind locations
    return ecoinvent_eff / remind_eff

def find_gas_efficiency_scaling_factor(ds, year, remind_efficiency, agg_func=np.average):
    #input a gas electricity dataset and year. We look up the efficiency for this region and year from the remind model and return the scaling factor by which to multiply all efficiency dependent exchanges.
    #If the ecoinvent region corresponds to multiple remind regions we simply average them.
    ecoinvent_eff = find_ecoinvent_gas_efficiency(ds)
    remind_locations= ecoinvent_to_remind_locations(ds['location'])
    remind_eff = agg_func(remind_efficiency [remind_locations].values)/100 # we take an average of all applicable remind locations
    return ecoinvent_eff / remind_eff

def find_oil_efficiency_scaling_factor(ds, year, remind_efficiency, agg_func=np.average):
    #input a oil electricity dataset and year. We look up the efficiency for this region and year from the remind model and return the scaling factor by which to multiply all efficiency dependent exchanges.
    #If the ecoinvent region corresponds to multiple remind regions we simply average them.
    ecoinvent_eff = find_ecoinvent_oil_efficiency(ds)
    remind_locations= ecoinvent_to_remind_locations(ds['location'])
    remind_eff = agg_func(remind_efficiency [remind_locations].values)/100 # we take an average of all applicable remind locations
    return ecoinvent_eff / remind_eff

def find_biomass_efficiency_scaling_factor(ds, year, remind_efficiency, agg_func=np.average):
    #input an electricity dataset and year. We look up the efficiency for this region and year from the remind model and return the scaling factor by which to multiply all efficiency dependent exchanges.
    #If the ecoinvent region corresponds to multiple remind regions we simply average them.
    ecoinvent_eff = find_ecoinvent_biomass_efficiency(ds)
    remind_locations= ecoinvent_to_remind_locations(ds['location'])
    remind_eff = agg_func(remind_efficiency [remind_locations].values)/100 # we take an average of all applicable remind locations
    return ecoinvent_eff / remind_eff

## Get remind emissions

In [87]:
def get_emission_factors(scenario = "SSP2"):
    gains_emi = pd.read_csv("GAINS emission factors.csv", skiprows=4, 
                            names=["year", "region", "GAINS", "pollutant", "scenario", "factor"])
    gains_emi["unit"] = "Mt/TWa"
    gains_emi = gains_emi[gains_emi.scenario == scenario]

    sector_mapping = pd.read_csv("GAINStoREMINDtechmap.csv").drop(["noef", "elasticity"], axis=1)
    
    return gains_emi.join(sector_mapping.set_index("GAINS"), on="GAINS").dropna().drop(['scenario', 'REMIND'], axis=1).set_index(['year', 'region','GAINS', 'pollutant'])['factor'].unstack(level = [0,1,3]) /8760 #kg / kWh


# unfortunately, we currently don't have much resolution in these technologies, but it's better than nothing:
emissions_lookup_dict = {
    'Biomass IGCC CCS':'Power_Gen_Bio_Trad',
    'Biomass IGCC': 'Power_Gen_Bio_Trad',
    'Biomass CHP':'Power_Gen_Bio_Trad',
    'Coal IGCC':'Power_Gen_Coal' ,
    'Coal IGCC CCS':'Power_Gen_Coal' ,
    'Coal PC':    'Power_Gen_Coal'     , 
    'Coal CHP': 'Power_Gen_Coal' ,
    'Coal PC CCS':'Power_Gen_Coal' ,
    'Gas CCS':'Power_Gen_NatGas',
    'Gas OC':  'Power_Gen_NatGas'   , 
    'Gas CC':  'Power_Gen_NatGas',
    'Gas CHP':'Power_Gen_NatGas',
    'Oil': 'Power_Gen_LLF'}


def get_remind_emissions(remind_emissions_factors, year, region, tech):
    if year in remind_emissions_factors.columns.levels[0]: 
        result = remind_emissions_factors.loc[emissions_lookup_dict[tech]].unstack(level=[1,2])[region].loc[year]
    else:    
        temp = remind_emissions_factors.loc[emissions_lookup_dict[tech]].unstack(level=[1,2])[region]
        new = pd.DataFrame(index = temp.columns,columns = [year],  data = np.nan).T
        
        result =  pd.concat([temp, new]).sort_index().interpolate(method = 'values').loc[year]
        
    if type(region) == list:
        result = result.unstack(level=0)
        
    return result

## Modify ecoinvent fossil electricity generation technologies

In [88]:
remind_air_pollutants ={ # for now we don't have any emissions data from remind, so we scale everything by efficiency.
    'Sulfur dioxide': 'SO2', 
    'Carbon monoxide, fossil': 'CO', 
    'Nitrogen oxides': 'NOx',
    'Ammonia': 'NH3',
    'NMVOC, non-methane volatile organic compounds, unspecified origin': 'VOC',
    #'BC',
    #'OC',
}   

#define filter functions that decide which ecoinvent processes to modify
no_al = [exclude(contains('name', 'aluminium industry'))]
no_ccs = [exclude(contains('name', 'carbon capture and storage'))]
no_markets = [exclude(contains('name', 'market'))]
no_imports = [exclude(contains('name', 'import'))]
generic_excludes = no_al + no_ccs + no_markets

remind_mapping = {
    'Coal PC': {  
        'eff_func': find_coal_efficiency_scaling_factor,
        'technology filters': coal_electricity + generic_excludes,
        'technosphere excludes': [], # which technosphere exchanges to not change at all
    },
    'Coal CHP': {
       'eff_func': find_coal_efficiency_scaling_factor,
        'technology filters': coal_chp_electricity + generic_excludes,
        'technosphere excludes': [],  # which technosphere exchanges to not change at all      
    },
    'Gas OC': {
        'eff_func': find_gas_efficiency_scaling_factor,
        'technology filters': gas_open_cycle_electricity + generic_excludes + no_imports,
        'technosphere excludes': [],  # which technosphere exchanges to not change at all              
    },
    'Gas CC': {
        'eff_func': find_gas_efficiency_scaling_factor,
        'technology filters': gas_combined_cycle_electricity + generic_excludes + no_imports, 
        'technosphere excludes': [],  # which technosphere exchanges to not change at all              
    },
    'Gas CHP': {
        'eff_func': find_gas_efficiency_scaling_factor,
        'technology filters': gas_chp_electricity + generic_excludes + no_imports, 
        'technosphere excludes': [],    # which technosphere exchanges to not change at all            
    },    
    'Oil': {  
        'eff_func': find_oil_efficiency_scaling_factor,
        'technology filters': oil_open_cycle_electricity + generic_excludes+ [exclude(contains('name', 'nuclear'))],
        'technosphere excludes': [],# which technosphere exchanges to not change at all
    },  

#    'Biomass ST': {  
#        'eff_func': find_biomass_efficiency_scaling_factor,
#        'technology filters': biomass_electricity + generic_excludes,
#        'technosphere excludes': [],# which technosphere exchanges to not change at all
#    }, 
    'Biomass CHP': {  
        'eff_func': find_biomass_efficiency_scaling_factor,
        'technology filters': biomass_chp_electricity + generic_excludes,
        'technosphere excludes': [],# which technosphere exchanges to not change at all
    }, 
#    'Biomass CC': {  
#        'eff_func': find_biomass_efficiency_scaling_factor,
#        'technology filters': biomass_combined_cycle_electricity + generic_excludes,
#        'technosphere excludes': [],# which technosphere exchanges to not change at all
#    }, 

}

In [89]:
def update_electricity_datasets_with_remind_data(db, remind_data, year, agg_func=np.average, update_efficiency = True, update_emissions = True):
    """    
       This function modifies each ecoinvent coal, gas, oil and biomass dataset using data from the remind model. 
    """
    print("Don't forget that we aren't modifying PM emissions!")
    
    changes ={}
    
    for remind_technology in remind_mapping:
        print('Changing ', remind_technology)
        md = remind_mapping[remind_technology]
        remind_efficiency = get_remind_fossil_electricity_efficiency(remind_data, year, remind_technology)
        remind_emissions_factors = get_emission_factors()

        for ds in get_many(db, *md['technology filters']):
            changes[ds['code']]={}
            changes[ds['code']].update( {('meta data', x) : ds[x] for x in ['name','location']})
            changes[ds['code']].update( {('meta data', 'remind technology') : remind_technology})
            changes[ds['code']].update( {('original exchanges', k) :v for k,v in get_exchange_amounts(ds).items()})
            if update_efficiency == True:
                # Modify using remind efficiency values: 
                scaling_factor = md['eff_func'](ds, year, remind_efficiency, agg_func)
                update_ecoinvent_efficiency_parameter(ds, scaling_factor)
                change_exchanges_by_constant_factor(ds, scaling_factor, md['technosphere excludes'], 
                                                [doesnt_contain_any('name', remind_air_pollutants)])
            
            # we use this bit of code to explicitly rewrite the value for certain emissions.
            if update_emissions == True: 
                # Modify using remind specific emissions data
                remind_locations = ecoinvent_to_remind_locations(ds['location'])
                remind_emissions = get_remind_emissions(remind_emissions_factors, year, remind_locations, remind_technology)
                for exc in biosphere(ds, either(*[contains('name', x) for x in remind_air_pollutants])):
                    
                    flow = remind_air_pollutants[exc['name']]
                    amount =  agg_func(remind_emissions.loc[flow].values)
                        
                    #if new amount isn't a number:
                    if np.isnan(amount): 
                        print('Not a number! Setting exchange to zero' + ds['name'], exc['name'], ds['location'])
                        rescale_exchange(exc, 0) 
                        
                    #if old amound was zero:
                    elif exc['amount'] ==0:
                        exc['amount'] = 1 
                        rescale_exchange(exc, amount / exc['amount'], remove_uncertainty = True)
                        
                    else: 
                        rescale_exchange(exc, amount / exc['amount'])
 
            changes[ds['code']].update( {('updated exchanges', k) :v for k,v in get_exchange_amounts(ds).items()}) 
            
        #check if exchange amounts are real numbers:
            for k,v in get_exchange_amounts(ds).items():
                if np.isnan(v): print(ds, k)
    return changes

# Modifying Carma datasets

In [90]:
carma_electricity_ds_name_dict = {  
 'Electricity, at BIGCC power plant 450MW, no CCS/2025': 'Biomass IGCC',
    
 'Electricity, at BIGCC power plant 450MW, pre, pipeline 200km, storage 1000m/2025': 'Biomass IGCC CCS',
 'Electricity, at wood burning power plant 20 MW, truck 25km, post, pipeline 200km, storage 1000m/2025': 'Biomass IGCC CCS',
 'Electricity, from CC plant, 100% SNG, truck 25km, post, pipeline 200km, storage 1000m/2025': 'Biomass IGCC CCS',  

 'Electricity, at power plant/hard coal, IGCC, no CCS/2025': 'Coal IGCC',
 'Electricity, at power plant/lignite, IGCC, no CCS/2025': 'Coal IGCC',
 'Electricity, at power plant/hard coal, pre, pipeline 200km, storage 1000m/2025': 'Coal IGCC CCS',
 'Electricity, at power plant/lignite, pre, pipeline 200km, storage 1000m/2025': 'Coal IGCC CCS',
    
 'Electricity, at power plant/hard coal, post, pipeline 200km, storage 1000m/2025': 'Coal PC CCS',
 'Electricity, at power plant/lignite, post, pipeline 200km, storage 1000m/2025': 'Coal PC CCS',
    
 'Electricity, at power plant/natural gas, post, pipeline 200km, storage 1000m/2025': 'Gas CCS',
 'Electricity, at power plant/natural gas, pre, pipeline 200km, storage 1000m/2025': 'Gas CCS'}

In [91]:
def modify_all_carma_electricity_datasets(db, remind_data, year, update_efficiency = True, update_emissions = True):
    remind_emissions_factors = get_emission_factors()

    
    for name, remind_technology in carma_electricity_ds_name_dict.items():        
        remind_efficiency = get_remind_fossil_electricity_efficiency(remind_data, year, remind_technology)

        
        for ds in get_many(db, equals('name', name)):
            
            if update_efficiency: 
                if 'Electricity, at BIGCC power plant 450MW' in ds['name']: 
                    modify_carma_BIGCC_efficiency(ds, remind_efficiency)
                else:
                    modify_standard_carma_dataset_efficiency(ds, remind_efficiency)
            if update_emissions:             
                modify_carma_dataset_emissions(db, ds, remind_emissions_factors, year, remind_technology)
    
    
    
    #The efficiency defined by image also includes the electricity consumed in the carbon capture process, so we have to set this exchange amount to zero:
    if update_efficiency:
        for ds in get_many(db, contains('name', 'CO2 capture')):
            for exc in technosphere(ds, *[contains('name', 'Electricity'), equals('unit', 'kilowatt hour')]):
                exc['amount'] = 0
    return

In [92]:
def modify_carma_dataset_emissions(db, ds, remind_emissions_factors, year, remind_technology):
    #The dataset passed to this function doesn't have the biosphere flows directly. Rather, it has an exchange (with unit MJ) that contains the biosphere flows per unit fuel input. 

    biosphere_mapping={'SO2':'Sulfur dioxide', 
                       'CO': 'Carbon monoxide, fossil', 
                       'NOx': 'Nitrogen oxides',
                       } 
    
    remind_locations = ecoinvent_to_remind_locations(ds['location'])
    remind_emissions = get_remind_emissions(remind_emissions_factors, year, remind_locations, remind_technology)
        
    exc_dataset_names = [x['name'] for x in technosphere(ds, equals('unit', 'megajoule'))]
    
    for exc_dataset in get_many(db, *[either(*[equals('name', exc_dataset_name) for exc_dataset_name in exc_dataset_names])]):
        
        if len(list(biosphere(exc_dataset)))==0: 
            modify_carma_dataset_emissions(db, exc_dataset, remind_emissions_factors, year, remind_technology)
            continue
            
        #Modify using IMAGE emissions data
        for key, value in biosphere_mapping.items():
            for exc in biosphere(exc_dataset, contains('name', value)):          
                exc['amount'] = np.average(remind_emissions.loc[key][remind_locations])
                if np.isnan(exc['amount']): 
                    print('Not a number! Setting exchange to zero' + ds['name'], exc['name'], ds['location'])
                    exc['amount']=0
    return

In [93]:
def modify_carma_BIGCC_efficiency(ds, remind_efficiency):
    remind_locations = ecoinvent_to_remind_locations(ds['location'])
    remind_efficiency = np.average(remind_efficiency[remind_locations])
    
    old_efficiency = 3.6/get_one(technosphere(ds), *[contains('name', 'Hydrogen, from steam reforming')])['amount'] 

    for exc in technosphere(ds):
        exc['amount'] = exc['amount']*old_efficiency/remind_efficiency
        return

In [94]:
def modify_standard_carma_dataset_efficiency(ds, remind_efficiency):
    if 'Electricity, at BIGCC power plant 450MW' in ds['name']:
        print("This function can't modify dataset: ",ds['name'], "It's got a different format.")
        return
    
    remind_locations = ecoinvent_to_remind_locations(ds['location'])
    remind_efficiency = np.average(remind_efficiency[remind_locations])
    
    #All other carma electricity datasets have a single exchange that is the combustion of a fuel in MJ. 
    #We can just scale this exchange and efficiency related changes will be done
    
    for exc in technosphere(ds):
        exc['amount'] = 3.6/remind_efficiency
   
    return   